In [60]:
import qiskit
import numpy as np
from qiskit_ibm_runtime.fake_provider import FakeMumbaiV2
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import qiskit.quantum_info as qi

# Import from Qiskit Aer noise module
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error,coherent_unitary_error)

In [61]:
from qiskit import QuantumCircuit

def circular_ansatz_mirrored(N, reps=1, fix_2q=False): 
    qc = qiskit.QuantumCircuit(N)
    for _ in range(reps):
        for i in range(N):
            qc.ry(np.pi/2, i)
        for i in range(N):
            qc.rz(np.pi/2, i)
        for i in range(N):
            control = (i-1) % N
            target = i
            qc.cx(control, target)
        for i in range(N):
            qc.ry(np.pi/2, i)
        for i in range(N):
            qc.rz(np.pi/2, i)
        for i in range(N-1, -1, -1):
            control = (i-1) % N
            target = i
            qc.cx(control, target)
    for i in range(N):
        qc.ry(np.pi/2, i)
    for i in range(N):
        qc.rz(np.pi/2, i)
    return qc

def populated_circuit(n):
    qc = QuantumCircuit(n)
    qc.h(0)
    for i in range(n - 1):
        qc.cx(i, i + 1)
        if i + 2 < n:
            qc.cz(i + 1, i + 2)
    return qc


In [62]:
num_qubits = 5

np.random.seed(0)
paulis = ["".join(np.random.choice(['I', 'X', 'Y', 'Z'], size=num_qubits)) for _ in range(6)]
print(paulis[0])


qc = circular_ansatz_mirrored(num_qubits)
# qc = populated_circuit(num_qubits)


IZXIZ


In [69]:
#Fake Backend
backend = FakeMumbaiV2()
ideal_estimator = Estimator(mode=backend)
ideal_estimator.options.seed_estimator=0

# # Or define a real backend
# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService()
# backend = service.least_busy(operational=True, simulator=False)
# ideal_estimator = Estimator(mode=backend)
# print(backend)


In [70]:
# ZX means Z on the first qubit and X on the second qubit
np.random.seed(0)
coeffs = np.random.random(len(paulis))
observable = SparsePauliOp.from_list(list(zip(paulis, coeffs)))

In [71]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1,seed_transpiler=0)
isa_circuit = pm.run(qc)
isa_observable = observable.apply_layout(isa_circuit.layout)

In [72]:
job = ideal_estimator.run([(isa_circuit, isa_observable)])

/home/dbharadwaj/.local/lib/python3.10/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:233: UserWarning: Options {'seed_estimator': 0} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


In [67]:
# Get results for the first (and only) PUB
ideal_res = job.result()[0]
print(f">>> Expectation value: {ideal_res.data.evs}")

>>> Expectation value: 0.014005165165240365


## Noisy

In [48]:
noisy_estimator = Estimator(mode=backend)
noise_model = NoiseModel()

#Depolarizing Noise
# cx_depolarizing_prob = 0.1
# noise_model.add_all_qubit_quantum_error(
#     depolarizing_error(cx_depolarizing_prob, 2), ["cx"]
# )

#Coherent Noise
epsilon = 0.1

err_cx = QuantumCircuit(2)
err_cx.cx(0,1)
err_cx.p(epsilon, 0)
err_cx.p(epsilon, 1)

err_cx.cx(0,1)
err_cx.p(-epsilon, 0)
err_cx.p(-epsilon, 1)


err_cx = qi.Operator(err_cx)
noise_model.add_all_qubit_quantum_error(
    coherent_unitary_error(err_cx), ["cx"]
)

(noisy_estimator.options.simulator.noise_model) = noise_model

In [ ]:
# Get results for the first (and only) PUB

job = noisy_estimator.run([(isa_circuit, isa_observable)])
noisy_res = job.result()[0]

print(f">>> Expectation value: {noisy_res.data.evs}")
 

>>> Expectation value: -0.025211753660933485


In [50]:
abs_error_diff = abs(noisy_res.data.evs - ideal_res.data.evs)
print(f"Absolute error difference: {abs_error_diff}")

Absolute error difference: 0.004528080826400809


## Pauli Twirling Mitigation

In [51]:
pt_estimator = Estimator(mode=backend)

# Or define a real backend
# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService()
# backend = service.least_busy(operational=True, simulator=False)

pt_estimator.options.twirling.enable_gates = True
pt_estimator.options.twirling.num_randomizations = 32
pt_estimator.options.twirling.shots_per_randomization = 128
pt_estimator.options.simulator.noise_model = noise_model

In [13]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)
isa_observable = observable.apply_layout(isa_circuit.layout)

In [14]:
job = pt_estimator.run([(isa_circuit, isa_observable)])

/home/dbharadwaj/.local/lib/python3.10/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:233: UserWarning: Options {'twirling': {'enable_gates': True, 'num_randomizations': 32, 'shots_per_randomization': 128}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


In [15]:
# Get results for the first (and only) PUB
pt_res = job.result()[0]
print(f">>> Expectation value: {pt_res.data.evs}")

>>> Expectation value: -0.023875896300918818


In [16]:
abs_error_diff_pt = abs(pt_res.data.evs - ideal_res.data.evs)
print(f"Absolute error difference: {abs_error_diff_pt}")

Absolute error difference: 0.03802469216553721


In [17]:
factor_of_difference = abs_error_diff / abs_error_diff_pt
print(f"Factor of Improvement: {factor_of_difference}")

Factor of Improvement: 0.34976357563660565
